# main

In [1]:
import pandas as pd
from pathlib import Path
import json
from generate_dpe_annexes_scripts import td001_processing
from generate_dpe_annexes_scripts.td001_processing import postprocessing_td001
from generate_dpe_annexes_scripts.utils import round_float_cols, unique_ordered
from config import paths
from multiprocessing import Pool
from generate_dpe_annexes_scripts.td007_processing import merge_td007_tr_tv, postprocessing_td007, generate_pb_table, \
    generate_ph_table, generate_mur_table, agg_td007_mur_to_td001, agg_td007_ph_to_td001, agg_td007_pb_to_td001

from generate_dpe_annexes_scripts.td008_processing import merge_td008_tr_tv, postprocessing_td008
from generate_dpe_annexes_scripts.td001_merge import merge_td001_dpe_id_envelope
from generate_dpe_annexes_scripts.td007_processing import agg_td007_to_td001_essential, agg_surf_envelope
from generate_dpe_annexes_scripts.td008_processing import agg_td008_to_td001_essential, agg_td008_to_td001
from generate_dpe_annexes_scripts.td010_processing import merge_td010_tr_tv, postprocessing_td010, agg_td010_td001
from generate_dpe_annexes_scripts.td011_td012_processing import merge_td012_tr_tv, postprocessing_td012, merge_td011_tr_tv, \
    agg_systeme_chauffage_essential
from generate_dpe_annexes_scripts.td013_td014_processing import merge_td013_tr_tv, postprocessing_td014, merge_td014_tr_tv, \
    agg_systeme_ecs_essential
from generate_dpe_annexes_scripts.td001_merge import merge_td001_dpe_id_system
from generate_dpe_annexes_scripts.doc_annexe import td001_annexe_enveloppe_agg_desc, td001_sys_ch_agg_desc, td001_sys_ecs_agg_desc, \
    td007_annexe_desc, td008_annexe_desc, td012_annexe_desc, td014_annexe_desc, enums_cstb, \
    td001_annexe_generale_desc
from generate_dpe_annexes_scripts.td006_processing import agg_td006_td001,merge_td006_tr_tv




def run_enveloppe_processing(td001, td006, td007, td008, td010):
    td008_raw_cols = td008.columns.tolist()
    td007_raw_cols = td007.columns.tolist()
    td010_raw_cols = td010.columns.tolist()

    td001, td006, td007, td008, td010 = merge_td001_dpe_id_envelope(td001=td001, td006=td006, td007=td007, td008=td008,
                                                                    td010=td010)
    # POSTPRO DES TABLES
    td006 = merge_td006_tr_tv(td006)

    td008 = merge_td008_tr_tv(td008)
    td008 = postprocessing_td008(td008)

    td007 = merge_td007_tr_tv(td007)
    td007 = postprocessing_td007(td007, td008)

    td010 = merge_td010_tr_tv(td010)
    td010 = postprocessing_td010(td010)

    # TABLES PAR TYPE COMPOSANT
    td007_pb = generate_pb_table(td007)
    td007_ph = generate_ph_table(td007)
    td007_murs = generate_mur_table(td007)

    # TABLES SYNTHETIQUES TOUTES THEMATIQUES

    td007_agg_essential = agg_td007_to_td001_essential(td007)
    td008_agg_essential = agg_td008_to_td001_essential(td008)
    surfaces_agg_essential = agg_surf_envelope(td007, td008)

    td001_enveloppe_agg = pd.concat([td007_agg_essential, td008_agg_essential, surfaces_agg_essential], axis=1)

    td001_enveloppe_agg.index.name = 'td001_dpe_id'
    cols = [el for el in td008.columns if el not in td008_raw_cols + ['fen_lib_from_tv009',
                                                                      'fen_lib_from_tv021']]
    cols.append('td008_baie_id')
    cols = unique_ordered(cols)
    td008_p = td008[cols]
    cols = [el for el in td007.columns if
            el not in td007_raw_cols + ["qualif_surf", 'surf_paroi_opaque_calc', 'surf_paroi_totale_calc_v1',
                                        'surf_paroi_totale_calc_v2']]
    cols.append('td007_paroi_opaque_id')
    cols = unique_ordered(cols)
    td007_p = td007[cols]

    cols = [el for el in td010.columns if
            el not in td010_raw_cols]
    cols.append('td010_pont_thermique_id')
    cols = unique_ordered(cols)
    td010_p = td010[cols]

    # TABLES AGGREGEES PAR TYPE COMPOSANT
    td007_murs_agg = agg_td007_mur_to_td001(td007_murs)
    td007_ph_agg = agg_td007_ph_to_td001(td007_ph)
    td007_pb_agg = agg_td007_pb_to_td001(td007_pb)
    td008_agg = agg_td008_to_td001(td008)
    td010_agg = agg_td010_td001(td010)
    td006_agg = agg_td006_td001(td006)


    env_compo_dict = dict(td007_paroi_opaque=td007_p,
                          td007_ph=td007_ph,
                          td007_pb=td007_pb,
                          td007_murs=td007_murs,
                          td008_baie=td008_p,
                          td010_pont_thermique=td010_p)

    env_compo_agg_dict = dict(td007_murs_agg=td007_murs_agg,
                              td007_ph_agg=td007_ph_agg,
                              td007_pb_agg=td007_pb_agg, td008_agg=td008_agg, td010_agg=td010_agg,td006_agg=td006_agg)

    return td001_enveloppe_agg, td008_p, td007_p, env_compo_dict, env_compo_agg_dict


def run_system_processing(td001, td006, td011, td012, td013, td014):
    td011_raw_cols = td011.columns.tolist()
    td012_raw_cols = td012.columns.tolist()
    td013_raw_cols = td013.columns.tolist()
    td014_raw_cols = td014.columns.tolist()
    td001, td006, td011, td012, td013, td014 = merge_td001_dpe_id_system(td001, td006, td011, td012, td013, td014)
    td011 = merge_td011_tr_tv(td011)
    td012 = merge_td012_tr_tv(td012)
    td013 = merge_td013_tr_tv(td013)
    td014 = merge_td014_tr_tv(td014)

    td012 = postprocessing_td012(td012)

    cols = [el for el in td011.columns if el not in td011_raw_cols]
    cols.append('td011_installation_chauffage_id')
    cols = unique_ordered(cols)
    td011_p = td011[cols]

    cols = [el for el in td012.columns if
            el not in td012_raw_cols + ['besoin_chauffage_infer', 'gen_ch_concat_txt_desc']]
    cols.append('td012_generateur_chauffage_id')
    cols = unique_ordered(cols)
    td012_p = td012[cols]

    td001_sys_ch_agg = agg_systeme_chauffage_essential(td001, td011, td012)

    td014 = postprocessing_td014(td013, td014,td001,td001_sys_ch_agg)

    cols = [el for el in td013.columns if el not in td013_raw_cols]
    cols.append('td013_installation_ecs_id')
    cols = unique_ordered(cols)
    td013_p = td013[cols]

    cols = [el for el in td014.columns if
            el not in td014_raw_cols + ['score_gen_ecs_lib_infer', 'gen_ecs_concat_txt_desc']]
    cols.append('td014_generateur_ecs_id')
    cols = unique_ordered(cols)
    td014_p = td014[cols]

    td001_sys_ecs_agg = agg_systeme_ecs_essential(td001, td013, td014)

    return td011_p, td012_p, td001_sys_ch_agg, td013_p, td014_p, td001_sys_ecs_agg


def build_doc(annexe_dir):
    doc_annexe = dict()
    doc_annexe['td001_annexe_generale'] = td001_annexe_generale_desc
    doc_annexe['td001_annexe_enveloppe_agg'] = td001_annexe_enveloppe_agg_desc
    doc_annexe['td001_sys_ch_agg'] = td001_sys_ch_agg_desc
    doc_annexe['td001_sys_ecs_agg'] = td001_sys_ecs_agg_desc
    doc_annexe['td007_annexe'] = td007_annexe_desc
    doc_annexe['td008_annexe'] = td008_annexe_desc
    doc_annexe['td012_annexe'] = td012_annexe_desc
    doc_annexe['td014_annexe'] = td014_annexe_desc

    with open(annexe_dir / 'doc_table_annexes_cstb.json', 'w', encoding='utf-8') as f:
        json.dump(doc_annexe, f, indent=4)

    with open(annexe_dir / 'enum_table_annexes_cstb.json', 'w', encoding='utf-8') as f:
        json.dump(enums_cstb, f, indent=4)


data_dir = paths['DPE_DEPT_PATH']
annexe_dir = paths['DPE_DEPT_ANNEXE_PATH']
annexe_dir = Path(annexe_dir)
annexe_dir.mkdir(exist_ok=True, parents=True)


def run_postprocessing_by_depts(dept_dir):
    print(dept_dir)
    annexe_dept_dir = annexe_dir / dept_dir.name
    annexe_dept_dir.mkdir(exist_ok=True, parents=True)
    # LOAD TABLES
    td007 = pd.read_csv(dept_dir / 'td007_paroi_opaque.csv', dtype=str)
    td006 = pd.read_csv(dept_dir / 'td006_batiment.csv', dtype=str)
    td001 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
    td008 = pd.read_csv(dept_dir / 'td008_baie.csv', dtype=str)
    td008 = td008.drop('td008_baie_id', axis=1)
    td010 = pd.read_csv(dept_dir / 'td010_pont_thermique.csv', dtype=str)

    # ENVELOPPE PROCESSING
    td001_enveloppe_agg, td008_p, td007_p, env_compo_dict, env_compo_agg_dict = run_enveloppe_processing(td001,
                                                                                                         td006,
                                                                                                         td007,
                                                                                                         td008,
                                                                                                         td010)

    round_float_cols(td001_enveloppe_agg).to_csv(annexe_dept_dir / 'td001_enveloppe_agg_annexe.csv')
    round_float_cols(td007_p).to_csv(annexe_dept_dir / 'td007_paroi_opaque_annexe.csv')
    round_float_cols(td008_p).to_csv(annexe_dept_dir / 'td008_baie_annexe.csv')
    for k, v in env_compo_dict.items():
        round_float_cols(v).to_csv(annexe_dept_dir / f'{k}_annexe.csv')

    for k, v in env_compo_agg_dict.items():
        round_float_cols(v).to_csv(annexe_dept_dir / f'td001_{k}_annexe.csv')

    # EMPTY MEMORY
    del td001_enveloppe_agg, td008_p, td007_p, env_compo_dict, env_compo_agg_dict
    del v
    del td007, td008, td010

    # SYSTEM PROCESSING

    td011 = pd.read_csv(dept_dir / 'td011_installation_chauffage.csv', dtype=str)
    td012 = pd.read_csv(dept_dir / 'td012_generateur_chauffage.csv', dtype=str)
    td013 = pd.read_csv(dept_dir / 'td013_installation_ecs.csv', dtype=str)
    td014 = pd.read_csv(dept_dir / 'td014_generateur_ecs.csv', dtype=str)

    td011_p, td012_p, td001_sys_ch_agg, td013_p, td014_p, td001_sys_ecs_agg = run_system_processing(td001, td006,
                                                                                                    td011, td012,
                                                                                                    td013, td014)
    round_float_cols(td001_sys_ch_agg).to_csv(annexe_dept_dir / 'td001_sys_ch_agg_annexe.csv')
    round_float_cols(td001_sys_ecs_agg).to_csv(annexe_dept_dir / 'td001_sys_ecs_agg_annexe.csv')
    round_float_cols(td011_p).to_csv(annexe_dept_dir / 'td011_installation_chauffage_annexe.csv')
    round_float_cols(td012_p).to_csv(annexe_dept_dir / 'td012_generateur_chauffage_annexe.csv')
    round_float_cols(td013_p).to_csv(annexe_dept_dir / 'td013_installation_ecs_annexe.csv')
    round_float_cols(td014_p).to_csv(annexe_dept_dir / 'td014_generateur_ecs_annexe.csv')
    # EMPTY MEMORY
    del td011_p, td012_p, td001_sys_ch_agg, td013_p, td014_p, td001_sys_ecs_agg
    del td011, td012, td013, td014

    # add td001 processing
    postprocessing_td001(td001)[['nom_methode_dpe_norm', 'id']].rename(columns={'id': 'td001_dpe_id'}).to_csv(
        annexe_dept_dir / 'td001_annexe_generale.csv')



build_doc(annexe_dir)
list_dir = list(Path(data_dir).iterdir())
firsts = [a_dir for a_dir in list_dir if not (annexe_dir / a_dir.name / 'td001_annexe_generale.csv').is_file()]
lasts = [a_dir for a_dir in list_dir if (annexe_dir / a_dir.name / 'td001_annexe_generale.csv').is_file()]
print(len(firsts), len(lasts))
list_dir = firsts + lasts

# list_dir.reverse()

for dept_dir in list_dir:
    print(dept_dir)
    annexe_dept_dir = annexe_dir / dept_dir.name
    annexe_dept_dir.mkdir(exist_ok=True, parents=True)
    # LOAD TABLES
    td007 = pd.read_csv(dept_dir / 'td007_paroi_opaque.csv', dtype=str)
    td006 = pd.read_csv(dept_dir / 'td006_batiment.csv', dtype=str)
    td001 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
    td008 = pd.read_csv(dept_dir / 'td008_baie.csv', dtype=str)
    td008 = td008.drop('td008_baie_id', axis=1)
    td010 = pd.read_csv(dept_dir / 'td010_pont_thermique.csv', dtype=str)
    # ENVELOPPE PROCESSING
    break
    td001_enveloppe_agg, td008_p, td007_p, env_compo_dict, env_compo_agg_dict = run_enveloppe_processing(td001,
                                                                                                    td006,
                                                                                                         td007,
                                                                                                         td008,
                                                                                                         td010)
    

    round_float_cols(td001_enveloppe_agg).to_csv(annexe_dept_dir / 'td001_enveloppe_agg_annexe.csv')
    round_float_cols(td007_p).to_csv(annexe_dept_dir / 'td007_paroi_opaque_annexe.csv')
    round_float_cols(td008_p).to_csv(annexe_dept_dir / 'td008_baie_annexe.csv')
    for k, v in env_compo_dict.items():
        round_float_cols(v).to_csv(annexe_dept_dir / f'{k}_annexe.csv')

    for k, v in env_compo_agg_dict.items():
        round_float_cols(v).to_csv(annexe_dept_dir / f'td001_{k}_annexe.csv')



    # SYSTEM PROCESSING

    td011 = pd.read_csv(dept_dir / 'td011_installation_chauffage.csv', dtype=str)
    td012 = pd.read_csv(dept_dir / 'td012_generateur_chauffage.csv', dtype=str)
    td013 = pd.read_csv(dept_dir / 'td013_installation_ecs.csv', dtype=str)
    td014 = pd.read_csv(dept_dir / 'td014_generateur_ecs.csv', dtype=str)

    td011_p, td012_p, td001_sys_ch_agg, td013_p, td014_p, td001_sys_ecs_agg = run_system_processing(td001, td006,
                                                                                                    td011, td012,
                                                                                                    td013, td014)
    round_float_cols(td001_sys_ch_agg).to_csv(annexe_dept_dir / 'td001_sys_ch_agg_annexe.csv')
    round_float_cols(td001_sys_ecs_agg).to_csv(annexe_dept_dir / 'td001_sys_ecs_agg_annexe.csv')
    round_float_cols(td011_p).to_csv(annexe_dept_dir / 'td011_installation_chauffage_annexe.csv')
    round_float_cols(td012_p).to_csv(annexe_dept_dir / 'td012_generateur_chauffage_annexe.csv')
    round_float_cols(td013_p).to_csv(annexe_dept_dir / 'td013_installation_ecs_annexe.csv')
    round_float_cols(td014_p).to_csv(annexe_dept_dir / 'td014_generateur_ecs_annexe.csv')

    # add td001 processing
    postprocessing_td001(td001)[['nom_methode_dpe_norm', 'id']].rename(columns={'id': 'td001_dpe_id'}).to_csv(
        annexe_dept_dir / 'td001_annexe_generale.csv')

        
    break

ModuleNotFoundError: No module named 'elasticsearch'

In [9]:
from config import paths
from pathlib import Path
import pandas as pd
data_dir = paths['DPE_DEPT_PATH']
annexe_dir = paths['DPE_DEPT_ANNEXE_PATH']
annexe_dir= Path(annexe_dir)
annexe_dir.mkdir(exist_ok=True, parents=True)
dept_dir=Path(data_dir)/'94'
annexe_dept_dir=annexe_dir/'94'

In [39]:
td001 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
td006 = pd.read_csv(dept_dir / 'td006_batiment.csv', dtype=str)


In [40]:
td007 = pd.read_csv(dept_dir / 'td007_paroi_opaque.csv', dtype=str)
td007 = td007.rename(columns = {'id':'td007_paroi_opaque_id'})

In [41]:
td007_p=pd.read_csv(annexe_dept_dir / 'td007_paroi_opaque_annexe.csv', dtype=str)


In [42]:
cols = ['td007_paroi_opaque_id']+[el for el in td007.columns if el not in td007_p]



In [57]:
td007_f=td007[cols].merge(td007_p,on='td007_paroi_opaque_id',how='left')

In [58]:
td007_f=td007_f.astype({'td007_paroi_opaque_id': 'str',
               'td006_batiment_id': 'str',
               'tr014_type_parois_opaque_id': 'str',
               'reference': 'str',
               'deperdition_thermique': 'float',
               'tv001_coefficient_reduction_deperditions_id': 'category',
               'tv002_local_non_chauffe_id': 'category',
               'coefficient_transmission_thermique_paroi': 'float',
               'coefficient_transmission_thermique_paroi_non_isolee': 'float',
               'tv003_umur_id': 'category',
               'tv004_umur0_id': 'category',
               'tv005_upb_id': 'category',
               'tv006_upb0_id': 'category',
               'tv007_uph_id': 'category',
               'tv008_uph0_id': 'category',
               'resistance_thermique_isolation': 'float',
               'epaisseur_isolation': 'float',
               'surface_paroi': 'float'})

In [59]:
td007_f = td007_f.merge(
    td006.rename(columns={'id':'td006_batiment_id'})[['td006_batiment_id','td001_dpe_id']]).merge(
    td001.rename(columns={'id':'td001_dpe_id'})[['td001_dpe_id',"annee_construction",'surface_habitable','usr_logiciel_id','usr_diagnostiqueur_id']])

In [76]:
td007_f[['td006_batiment_id','tr014_type_parois_opaque_id','coefficient_transmission_thermique_paroi_non_isolee','coefficient_transmission_thermique_paroi','tv004_umur','tv003_umur','epaisseur_isolation','resistance_thermique_isolation','tv003_umur_id','tv004_umur0_id']]

,td006_batiment_id,tr014_type_parois_opaque_id,coefficient_transmission_thermique_paroi_non_isolee,coefficient_transmission_thermique_paroi,tv004_umur,tv003_umur,epaisseur_isolation,resistance_thermique_isolation,tv003_umur_id,tv004_umur0_id
0,164,1,0.0,0.36,2.75,0.36,0.0,0.0,52,82
1,164,1,0.0,0.36,2.75,0.36,0.0,0.0,52,82
2,164,1,0.0,2.00,2.75,2.50,0.0,0.0,1,82
3,164,1,0.0,2.00,2.75,2.50,0.0,0.0,1,82
4,164,4,0.0,2.00,NaN,NaN,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
285982,5418796,1,NaN,2.00,2.00,2.00,0.0,0.0,43,1
285983,5419443,1,0.0,0.84,2.90,0.80,0.0,0.0,48,81
285984,5419443,4,0.0,2.00,NaN,NaN,0.0,0.0,NaN,NaN
285985,5419443,1,0.0,2.00,2.90,2.50,0.0,0.0,1,81


In [86]:
td007=td007_f

In [87]:
td007_mur = td007.loc[td007.tr014_type_parois_opaque_id.isin(['2', '1'])]
td007_pb = td007.loc[td007.tr014_type_parois_opaque_id == '3']
td007_ph = td007.loc[td007.tr014_type_parois_opaque_id == '4']

In [89]:
td007_mur[['td006_batiment_id','tr014_type_parois_opaque_id','coefficient_transmission_thermique_paroi_non_isolee','coefficient_transmission_thermique_paroi','tv004_umur','tv003_umur','epaisseur_isolation','resistance_thermique_isolation','tv003_umur_id','tv004_umur0_id']]

,td006_batiment_id,tr014_type_parois_opaque_id,coefficient_transmission_thermique_paroi_non_isolee,coefficient_transmission_thermique_paroi,tv004_umur,tv003_umur,epaisseur_isolation,resistance_thermique_isolation,tv003_umur_id,tv004_umur0_id
0,164,1,0.0,0.36,2.75,0.36,0.0,0.0,52,82
1,164,1,0.0,0.36,2.75,0.36,0.0,0.0,52,82
2,164,1,0.0,2.00,2.75,2.50,0.0,0.0,1,82
3,164,1,0.0,2.00,2.75,2.50,0.0,0.0,1,82
6,220,1,0.0,2.00,2.45,NaN,0.0,0.0,NaN,5
...,...,...,...,...,...,...,...,...,...,...
285980,5418796,1,NaN,0.40,2.90,0.40,0.0,0.0,51,81
285981,5418796,1,NaN,0.40,2.90,0.40,0.0,0.0,51,81
285982,5418796,1,NaN,2.00,2.00,2.00,0.0,0.0,43,1
285983,5419443,1,0.0,0.84,2.90,0.80,0.0,0.0,48,81


In [ ]:
td007_mur['u']=td007

In [85]:
(td007_mur.deperdition_thermique==0).mean()

0.10902730950775455

In [65]:
td007_f.loc[td007_f.deperdition_thermique==0].usr_logiciel_id.value_counts()

3     32101
10     4457
5      4353
16     1253
6      1134
9       341
7       153
12      138
11       25
13        9
14        8
2         1
Name: usr_logiciel_id, dtype: int64

In [61]:
td007_f.loc[td007_f.coefficient_transmission_thermique_paroi_non_isolee==0].usr_logiciel_id.value_counts()

3     86777
12     2950
5      2764
7      1904
6      1884
16      736
9       613
14      199
2       176
Name: usr_logiciel_id, dtype: int64

In [62]:
td001.usr_logiciel_id.value_counts()

3     53809
10    43195
2     14986
5     14957
16     6639
13     5720
11     2367
12     2233
7       813
6       728
9       340
14      159
4        31
15        2
Name: usr_logiciel_id, dtype: int64

In [273]:
td007_murs = pd.read_csv(annexe_dept_dir / 'td007_murs_annexe.csv', dtype=str)

In [274]:
td007_murs = td007_murs.merge(
    td006.rename(columns={'id':'td006_batiment_id'})[['td006_batiment_id','td001_dpe_id']]).merge(
    td001.rename(columns={'id':'td001_dpe_id'})[['td001_dpe_id',"annee_construction",'surface_habitable','usr_logiciel_id','usr_diagnostiqueur_id']])

In [275]:
td007_murs=td007_murs.astype({'td007_paroi_opaque_id': 'str',
               'td006_batiment_id': 'str',
               'tr014_type_parois_opaque_id': 'str',
               'reference': 'str',
               'deperdition_thermique': 'float',
               'tv001_coefficient_reduction_deperditions_id': 'category',
               'tv002_local_non_chauffe_id': 'category',
               'coefficient_transmission_thermique_paroi': 'float',
               'coefficient_transmission_thermique_paroi_non_isolee': 'float',
               'tv003_umur_id': 'category',
               'tv004_umur0_id': 'category',
               'tv005_upb_id': 'category',
               'tv006_upb0_id': 'category',
               'tv007_uph_id': 'category',
               'tv008_uph0_id': 'category',
               'resistance_thermique_isolation': 'float',
               'epaisseur_isolation': 'float',
               'surface_paroi': 'float'})
float_cols = ['tv004_umur','annee_construction','annee_isole_uniforme_min']
td007_murs[float_cols]=td007_murs[float_cols].astype(float)

In [276]:

td007_murs['u']=td007_murs['coefficient_transmission_thermique_paroi'].astype(float)
td007_murs['u0']=td007_murs['coefficient_transmission_thermique_paroi_non_isolee'].astype(float)
td007_murs['u0']=td007_murs[['u0','tv004_umur']].max(axis=1)
u_2=td007_murs.u==2
td007_murs.loc[u_2,'u']=td007_murs.loc[u_2,'u0']

In [277]:
td007_murs['annee_construction']=td007_murs.annee_construction.astype(float)
td007_murs['annee_isole_uniforme_min']=td007_murs.annee_isole_uniforme_min.astype(float)

In [304]:
is_renove=td007_murs.annee_isole_uniforme_min>td007_murs.annee_construction+20
is_renove=is_renove&(td007_murs.annee_isole_uniforme_min>2000)
is_null = td007_murs.annee_isole_uniforme_min.isnull()
is_renove_null =is_null&(td007_murs.annee_construction<1988)&(td007_murs.u<0.4)
is_renove = is_renove |is_renove_null
td007_murs['is_renove']=is_renove

In [305]:
td007_murs.query('is_renove==True and meth_calc_isolation=="ISOLE DEFAUT PRE 1988"')[['u','coefficient_transmission_thermique_paroi','epaisseur_isolation','annee_construction','annee_isole_uniforme_min']]

,u,coefficient_transmission_thermique_paroi,epaisseur_isolation,annee_construction,annee_isole_uniforme_min


In [306]:
td007_murs.query('is_renove==True').meth_calc_isolation.value_counts()

ISOLE DEFAUT POST 1988      23806
ISOLE SAISI                  8757
STRUCTURE ISOLANTE (ITR)        4
Name: meth_calc_isolation, dtype: int64

In [292]:
td007_murs.loc[td007_murs.usr_diagnostiqueur_id=="7337"][['coefficient_transmission_thermique_paroi','epaisseur_isolation']]

,coefficient_transmission_thermique_paroi,epaisseur_isolation
21729,0.34,NaN
21730,0.34,NaN
21731,0.34,NaN
21732,0.34,NaN
21733,2.00,NaN
...,...,...
73048,2.99,NaN
73049,2.99,NaN
73050,3.45,NaN
73051,3.45,NaN


In [286]:
(1/(0.04/10))

250.0

In [281]:
td007_murs.loc[is_renove_null][["u","u0",'epaisseur_isolation','resistance_thermique_isolation']]

,u,u0,epaisseur_isolation,resistance_thermique_isolation
126,0.35,2.65,10.0,NaN
127,0.35,2.65,10.0,NaN
128,0.35,2.65,10.0,NaN
129,0.35,2.65,10.0,NaN
130,0.35,2.65,10.0,NaN
...,...,...,...,...
182964,0.29,2.40,12.0,NaN
182965,0.26,2.40,14.0,NaN
182966,0.26,2.40,14.0,NaN
182967,0.29,2.40,12.0,NaN


In [173]:
td007_murs.loc[(td007_murs.annee_construction==1974)&(td007_murs.meth_calc_isolation=='ISOLE DEFAUT POST 1988')][['is_renove','meth_calc_u','meth_calc_isolation','annee_construction','annee_isole_uniforme_min','annee_isole_uniforme_max','tv003_annee_construction_min','tv003_annee_construction_max','tv003_annee_isolation_min','tv003_annee_isolation_max']]

,is_renove,meth_calc_u,meth_calc_isolation,annee_construction,annee_isole_uniforme_min,annee_isole_uniforme_max,tv003_annee_construction_min,tv003_annee_construction_max,tv003_annee_isolation_min,tv003_annee_isolation_max
40,True,PAR DEFAUT PERIODE : ISOLE,ISOLE DEFAUT POST 1988,1974.0,2001.0,2900,2001,2900,NaN,NaN
41,True,PAR DEFAUT PERIODE : ISOLE,ISOLE DEFAUT POST 1988,1974.0,2001.0,2900,2001,2900,NaN,NaN
42,True,PAR DEFAUT PERIODE : ISOLE,ISOLE DEFAUT POST 1988,1974.0,2001.0,2900,2001,2900,NaN,NaN
43,True,PAR DEFAUT PERIODE : ISOLE,ISOLE DEFAUT POST 1988,1974.0,2001.0,2900,2001,2900,NaN,NaN
44,True,PAR DEFAUT PERIODE : ISOLE,ISOLE DEFAUT POST 1988,1974.0,2001.0,2900,2001,2900,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
182680,True,PAR DEFAUT PERIODE : ISOLE,ISOLE DEFAUT POST 1988,1974.0,2001.0,2900,2001,2900,NaN,NaN
182682,True,PAR DEFAUT PERIODE : ISOLE,ISOLE DEFAUT POST 1988,1974.0,2001.0,2900,2001,2900,NaN,NaN
182685,True,PAR DEFAUT PERIODE : ISOLE,ISOLE DEFAUT POST 1988,1974.0,2001.0,2900,2001,2900,NaN,NaN
182792,True,PAR DEFAUT PERIODE : ISOLE,ISOLE DEFAUT POST 1988,1974.0,2001.0,2900,2001,2900,NaN,NaN


In [111]:
td007_murs.annee_construction.value_counts().to_dict()

{'1974': 47592,
 '2006': 21045,
 '1982': 18878,
 '1948': 15456,
 '2000': 14210,
 '1989': 9293,
 '2005': 3973,
 '2001': 3781,
 '1977': 3777,
 '1983': 3577,
 '1988': 2892,
 '1978': 2359,
 '1969': 2137,
 '1975': 1755,
 '1970': 1726,
 '1960': 1504,
 '1995': 1488,
 '1967': 1484,
 '1950': 1400,
 '1965': 1151,
 '1990': 1141,
 '1980': 946,
 '2015': 924,
 '1985': 920,
 '1947': 897,
 '2008': 769,
 '1994': 760,
 '2011': 693,
 '2007': 686,
 '1976': 667,
 '2010': 665,
 '1981': 611,
 '1991': 550,
 '1996': 550,
 '1987': 546,
 '1955': 544,
 '1984': 539,
 '1992': 500,
 '2013': 482,
 '1986': 472,
 '1997': 467,
 '2009': 467,
 '2018': 466,
 '1999': 459,
 '1972': 450,
 '2003': 444,
 '2016': 411,
 '1993': 408,
 '2002': 407,
 '2004': 400,
 '1998': 395,
 '1968': 382,
 '2012': 319,
 '1964': 306,
 '1958': 285,
 '2017': 274,
 '2014': 268,
 '1979': 266,
 '1973': 264,
 '1971': 258,
 '1956': 241,
 '1954': 214,
 '1963': 202,
 '1957': 193,
 '1953': 187,
 '1962': 186,
 '1959': 174,
 '1900': 173,
 '1952': 166,
 '1966':

In [108]:
td007_murs.loc[td007_murs.annee_construction=="1974"].meth_calc_isolation.value_counts()

ISOLE DEFAUT POST 1988         24143
NON ISOLE                      19666
ISOLE DEFAUT PRE 1988           1781
ISOLE SAISI                     1105
ISOLATION INCONNUE (DEFAUT)      894
STRUCTURE ISOLANTE (ITR)           3
Name: meth_calc_isolation, dtype: int64